In [2]:
!pip3 install pandas boto3 mysql-connector-python scikit-learn numpy

In [5]:
import sys
from io import BytesIO
sys.path.append('../..') # Adiciona o diretório superior ao caminho de importação para acessar funções utilitárias
from functions.select_cols_functions import * # Importa todas as funções de manipulação de colunas
from functions.aws_functions import * # Importa todas as funções relacionadas à AWS
import util.credentials as u # Importa as credenciais da AWS de um módulo utilitário

In [6]:
bucket_name = 'adoptai-raw-dev' # Nome do bucket S3 de origem
new_bucket_name = 'adoptai-trusted-dev' # Nome do bucket S3 de destino

In [7]:
# Cria uma sessão boto3 usando as credenciais fornecidas
session = boto3.Session(
    aws_access_key_id= u.aws_access_key_id,
    aws_secret_access_key= u.aws_secret_access_key,
    aws_session_token= u.aws_session_token
)

s3_client = session.client('s3') # Inicializa o cliente S3 para interagir com os buckets

In [8]:
try:
    # Carrega o CSV mais recente do bucket S3 de origem e converte para um DataFrame
    s3_csv = s3_csv_to_df(bucket_name)
    
    object_key = s3_csv[0] # Obtém o nome do arquivo
    df = s3_csv[1] # Obtém o DataFrame do arquivo CSV

    # Aplica regras de limpeza e ajuste de dados ao DataFrame
    df_treated = apply_all_rules(df)

    # Converte o DataFrame tratado de volta para CSV e armazena em um buffer de memória
    csv_buffer = BytesIO()
    df_treated.to_csv(csv_buffer, index=False)

    # Salva o arquivo CSV tratado no bucket S3 de destino
    s3_client.put_object(Bucket=new_bucket_name, Key=object_key, Body=csv_buffer.getvalue())
    print(f'Novo arquivo salvo em {new_bucket_name}/{object_key}')
except Exception as e:
    # Captura e exibe qualquer erro que ocorra durante o processo
    print(f'Erro ao gravar o arquivo no s3 destino: {e}')

Último arquivo adicionado: train_2_classes.csv
Novo arquivo salvo em adoptai-trusted-prod/train_2_classes.csv
